In [24]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

events_data = pd.read_csv('static/First_contest//event_data_train.csv')
submissions_data = pd.read_csv('C:/Users/filip/ML/static/First_contest/submissions_data_train.csv')
events_data['date'] = pd.to_datetime(events_data.timestamp, unit='s')
events_data['day'] = events_data.date.dt.date

submissions_data['date'] = pd.to_datetime(submissions_data.timestamp, unit='s')
submissions_data['day'] = submissions_data.date.dt.date

users_data = events_data.groupby('user_id', as_index=False) \
    .agg({'timestamp': 'max'}) \
    .rename({'timestamp': 'last_timestamp'}, axis='columns')
now = 1526772811
drop_out_threasold = 2592000

users_data['is_gone_user'] = (now - users_data.last_timestamp) > drop_out_threasold
users_scores = submissions_data.pivot_table(index='user_id', \
                                            columns='submission_status', \
                                            values='step_id', \
                                            aggfunc='count', \
                                            fill_value=0) \
    .reset_index()
users_data = users_data.merge(users_scores, on='user_id', how='outer')
users_data = users_data.fillna(0)
users_invent_data = events_data.pivot_table(index='user_id',
                                            columns='action',
                                            values='step_id',
                                            aggfunc='count',
                                            fill_value=0).reset_index()
users_data = users_data.merge(users_invent_data, how='outer')
users_days = events_data.groupby('user_id').day.nunique()
users_days.to_frame().reset_index()
users_data = users_data.merge(users_days, on='user_id', how='outer')
users_data['passed_corse'] = users_data.passed > 170

submissions_data.step_id.mode()

submissions_data

events_data

submissions_data[submissions_data.submission_status == 'wrong'] \
    .groupby("step_id").agg({'submission_status': 'count'}). \
    sort_values('submission_status', ascending=True).head(10)

user_min_time = events_data.groupby('user_id', as_index=False). \
    agg({'timestamp': 'min'}) \
    .rename({"timestamp": 'min_timestamp'}, axis=1)
users_data = users_data.merge(user_min_time, how='outer')
users_data

events_data

event_data_train = pd.DataFrame()

events_data['user_time'] = events_data.user_id.map(str) \
                           + "_" + events_data.timestamp.map(str)

events_data

learning_time_threshold = 3 * 24 * 60 * 60

user_learning_time_threshold = user_min_time.user_id.map(str) \
                               + '_' + (learning_time_threshold + user_min_time.min_timestamp).map(str)

user_min_time["user_learning_time_threshold"] = user_learning_time_threshold
user_min_time

events_data = events_data \
    .merge(user_min_time[['user_id', 'user_learning_time_threshold']],
           how='outer')

events_data.shape

events_data_train = events_data[events_data.user_time <= events_data.user_learning_time_threshold]

events_data_train.groupby('user_id').day.nunique().max()

submissions_data['users_time'] = submissions_data.user_id.map(str) + '_' + submissions_data.timestamp.map(str)
submissions_data = submissions_data.merge(user_min_time[['user_id', 'user_learning_time_threshold']], how='outer')
submissions_data_train = submissions_data[submissions_data.users_time <= submissions_data.user_learning_time_threshold]
submissions_data_train.groupby('user_id').day.nunique().max()

X = submissions_data_train.groupby('user_id').day \
    .nunique().to_frame().reset_index().rename(columns={'day': 'days'})

steps_tried = submissions_data_train.groupby('user_id').step_id.nunique() \
    .to_frame().reset_index().rename(columns={'step_id': 'steps_tried'})

X = X.merge(steps_tried, on='user_id', how='outer')

X = X.merge(submissions_data_train.pivot_table(index='user_id', \
                                               columns='submission_status', \
                                               values='step_id', \
                                               aggfunc='count', \
                                               fill_value=0) \
            .reset_index())

X['correct_ratio'] = X.correct / (X.correct + X.wrong)

X

X = X.merge(events_data_train.pivot_table(index='user_id',
                                          columns='action', \
                                          values='step_id', \
                                          aggfunc='count', \
                                          fill_value=0) \
            .reset_index()[['user_id', 'viewed']], how='outer')

X = X.fillna(0)
users_data

X = X.merge(users_data[['user_id', 'passed_corse', 'is_gone_user']], how='outer')

X

X = X[~((X.is_gone_user == False) & (X.passed_corse == False))]

X.groupby(['passed_corse', 'is_gone_user']).user_id.count()

y = X.passed_corse.map(int)

X = X.drop(['passed_corse', 'is_gone_user'], axis=1)

X = X.set_index(X.user_id)
X = X.drop('user_id', axis=1)

# import pandas as pd
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
#
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# dt = DecisionTreeClassifier(criterion='entropy')
# parameters = {'max_depth': range(3, 6), 'max_leaf_nodes': range(6, 15), 'min_samples_leaf': range(1, 4),
#               'min_samples_split': range(2, 5)}
# grid_search_cv_clf = GridSearchCV(dt, parameters, cv=4)
# grid_search_cv_clf.fit(X_train, y_train)
# model = grid_search_cv_clf.best_estimator_
# print(grid_search_cv_clf.best_params_, cross_val_score(model, X_train, y_train, cv=4).mean())
#
# import pandas as pd
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
#
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# knn = KNeighborsClassifier()
# parameters = {'n_neighbors': range(15, 25), 'leaf_size': range(1, 7)}
# grid_search_cv_clf = GridSearchCV(knn, parameters, cv=4, n_jobs=-1)
# grid_search_cv_clf.fit(X_train, y_train)
# model = grid_search_cv_clf.best_estimator_
# print(grid_search_cv_clf.best_params_, cross_val_score(model, X_train, y_train, cv=4).mean())
#
# from sklearn.linear_model import LogisticRegressionCV
# from sklearn.model_selection import train_test_split
#
# clf = LogisticRegressionCV(cv=5)
#
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)
#
# clf.fit(X_train, y_train)
# print(clf.score(X_test, y_test))
# print(clf.coef_)
# print(clf.intercept_)
# print(cross_val_score(clf, X_train, y_train, cv=4).mean())


17985
17985
